In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
glassdoor_df = pd.read_csv('../data/google-amazon-facebook-employee-reviews/employee_reviews.csv', index_col=0)

In [3]:
glassdoor_df.drop(['link'], axis=1, inplace=True)

In [4]:
employee_titles_df = glassdoor_df["job-title"].str.split(" - ", n = 1, expand = True)
employee_titles_df.columns = ['Current-Employee', 'role']
employee_titles_df['is_current_employee'] = employee_titles_df['Current-Employee'].str.contains("Current Employee").astype(int)
employee_titles_df.drop(['Current-Employee'], axis=1, inplace=True)

In [5]:
employee_titles_df

,role,is_current_employee
1,Anonymous Employee,1
2,Program Manager,0
3,Software Engineer III,1
4,Anonymous Employee,1
5,Software Engineer,0
6,SDE2,0
7,Software Engineer,1
8,Anonymous Employee,0
9,Anonymous Employee,1
10,Anonymous Employee,1


In [6]:
glassdoor_df = pd.concat([glassdoor_df, employee_titles_df], axis=1)

In [7]:
glassdoor_df['year'] = glassdoor_df['dates'].str.split(', ').str[1]

In [8]:
for i, row in glassdoor_df.iterrows():
    if row['year'] == '0000' or pd.isnull(row['year']):
        glassdoor_df.drop([i], inplace=True)

In [9]:
glassdoor_df['month'] = glassdoor_df['dates'].str.split(', ').str[0].str.split(' ').str[1]

In [10]:
quarter_list = []
for _, row in glassdoor_df.iterrows():
    if row['month'] == 'Jan' or row['month'] == 'Feb' or row['month'] == 'Mar':
        quarter_list.append(1)
    elif row['month'] == 'Apr' or row['month'] == 'May' or row['month'] == 'Jun':
        quarter_list.append(2)
    elif row['month'] == 'Jul' or row['month'] == 'Aug' or row['month'] == 'Sep':
        quarter_list.append(3)
    elif row['month'] == 'Oct' or row['month'] == 'Nov' or row['month'] == 'Dec':
        quarter_list.append(4)
    else:
        print(row)
        quarter_list.append(5)

In [11]:
glassdoor_df['quarter'] = quarter_list

In [12]:
glassdoor_df.columns

Index(['company', 'location', 'dates', 'job-title', 'summary', 'pros', 'cons',
       'advice-to-mgmt', 'overall-ratings', 'work-balance-stars',
       'culture-values-stars', 'carrer-opportunities-stars',
       'comp-benefit-stars', 'senior-mangemnet-stars', 'helpful-count', 'role',
       'is_current_employee', 'year', 'month', 'quarter'],
      dtype='object')

In [14]:
glassdoor_df.columns = ['company', 'location', 'dates', 'job-title', 'summary', 'pros', 'cons',
       'advice-to-mgmt', 'overall-ratings', 'work-balance-stars',
       'culture-values-stars', 'career-opportunities-stars',
       'comp-benefit-stars', 'senior-management-stars', 'helpful-count', 'role', 'is_current_employee', 'year', 'month', 'quarter']

In [15]:
for i, row in glassdoor_df.iterrows():
    if row['overall-ratings'] == 'none':
        glassdoor_df.loc[i, 'overall-ratings'] = '-1'
    if row['work-balance-stars'] == 'none':
        glassdoor_df.loc[i, 'work-balance-stars'] = '-1'
    if row['culture-values-stars'] == 'none':
        glassdoor_df.loc[i, 'culture-values-stars'] = '-1'
    if row['career-opportunities-stars'] == 'none':
        glassdoor_df.loc[i, 'career-opportunities-stars'] = '-1'
    if row['comp-benefit-stars'] == 'none':
        glassdoor_df.loc[i, 'comp-benefit-stars'] = '-1'
    if row['senior-management-stars'] == 'none':
        glassdoor_df.loc[i, 'senior-management-stars'] = '-1'
        
    

In [20]:
star_list = ['overall-ratings', 'work-balance-stars', 'culture-values-stars', 'career-opportunities-stars', 'comp-benefit-stars', 'senior-management-stars']
for feature in star_list:
    glassdoor_df[feature] = glassdoor_df[feature].astype('float')

### Amazon Reviews + Their Quarterly Earnings

In [21]:
q_earnings_df = pd.read_csv('../data/amazon_quarterly_earnings.csv')
q_earnings_df['Year'] = q_earnings_df['Year'].astype('str')

In [22]:
amazon_reviews_df = glassdoor_df[glassdoor_df['company'] == 'amazon']

In [23]:
earnings_list = []
for _, row in amazon_reviews_df.iterrows():
    net_income = q_earnings_df['Quarterly Net Income (Billions)'].loc[(q_earnings_df['Quarter'] == row['quarter']) & (q_earnings_df['Year'] == row['year'])]
    if len(net_income.values) > 0:
        earnings_list.append(net_income.values[0])
    else:
        earnings_list.append(None)

In [24]:
amazon_reviews_df['amazon_earnings_this_quarter'] = earnings_list

C:\Users\jq\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
amazon_reviews_df['summary'].isna().any()

True

In [26]:
amazon_reviews_df['advice-to-mgmt'].isna().any()

True

Quarter starts: Jan. 1, Apr. 1, July 1 and Oct. 1.

In [ ]:

amazon_reviews_df['culture-values-stars'] = amazon_reviews_df['culture-values-stars'].astype('float')

In [27]:
y = amazon_reviews_df['work-balance-stars']

In [29]:
# Not going to use dates, job-title, role, overall-ratings, location 
# for now

X = amazon_reviews_df[['culture-values-stars', 'career-opportunities-stars', 
                       'comp-benefit-stars', 'senior-management-stars', 'helpful-count',
                       'is_current_employee', 'year', 'month', 'quarter', 'amazon_earnings_this_quarter']]

In [33]:
X

,culture-values-stars,career-opportunities-stars,comp-benefit-stars,senior-management-stars,helpful-count,is_current_employee,year,month,quarter,amazon_earnings_this_quarter
7821,5.0,5.0,4.0,5.0,0,1,2018,Dec,4,3.027
7822,5.0,5.0,5.0,5.0,1759,1,2016,Jan,1,0.513
7823,3.0,3.0,4.0,4.0,1404,1,2016,Feb,1,0.513
7824,3.0,5.0,5.0,4.0,1275,1,2013,Dec,4,0.240
7825,2.0,1.0,5.0,1.0,409,0,2017,Mar,1,0.724
7826,3.0,4.0,4.0,5.0,257,0,2017,May,2,0.197
7827,2.0,4.0,3.0,3.0,245,1,2017,Mar,1,0.724
7828,5.0,5.0,5.0,4.0,2,1,2018,Dec,4,3.027
7829,5.0,4.0,5.0,5.0,1,1,2018,Dec,4,3.027
7830,4.0,3.0,4.0,5.0,0,1,2018,Dec,4,3.027
